## database
#### Written by: F423894
#### Date: 03/12/24
### __Cell:__
database is used to import all libraries and files used throughout the entire program. It is also used to define common functions that are used in various other cells.
### __Functions:__
#### get_music_details():
Is called when specific data stored in "Music_Info.txt" needs to be read from and used in computations. Stores the data from the file in the form of lists.
#### get_rental_details(): 
Is called when specific data stored in "Rental.txt" needs to be read from and used in computations. Also stores the data contained within the file in lists.

In [1]:
#database
import subscriptionManager as sm
import feedbackManager as fm
import ipywidgets as widgets
from ipywidgets import *
import matplotlib.pyplot as plt
from datetime import datetime

def get_music_details():
    """Returns two lists: one contains categories, the other contains each records details"""
    f = open("Music_Info.txt", "r")
    music_categories = []
    categories = f.readline()
    for category in (categories.split(",")):
        music_categories.append(category.strip())

    music_details = []
    info = f.readlines()
    for details in info:
        music_details.append((details.strip()).split(","))
    f.close()
    return music_categories, music_details

def get_rental_details():
    """Same functionality as 'get_music_details()' but with the 'Rental.txt' file instead"""
    f = open("Rental.txt", "r")
    rental_categories = (f.readline()).split(",")

    rental_details = []
    rental_info = f.readlines()
    for details in rental_info:
        rental_details.append((details.strip()).split(","))
    f.close()
    return rental_categories, rental_details

## musicSearch
#### Written by:F423894
#### Date: 05/12/24
### __Cell:__
musicSearch is used to define the function called "music_search()", which takes inputs from the customer and cross-references these inputs with the "Music_Info.txt" and "Rental.txt" files using the functions defined in the database cell. For valid searches, it will produce a list of all records that match that search.
### __Functions:__
#### music_search(Music_Search, select_type, search_input):
Takes three parameters: 
 - "Music_Search" represents the widget that contains all other widgets that make up the GUI displayed under the "Music Search" tab. Is used to directly change widget values that display the list of records which match the users search.
 - "select_type" represents the value of the "select_type" widget. Is used to determine the category the user is searching under.
 - "search_input" represents the value of the "search_input" widget. Is used to get the customers specifc input from the text box (e.g. Name of Artist)

In [2]:
#music search
def music_search(Music_Search, select_type, search_input):
    """Searches through both 'Music_Info' and 'Rental' to output a list of specific records"""
    music_categories, music_details = get_music_details()
    rental_categories, rental_details = get_rental_details()

    search_type = select_type 
    #category that the user selected
    search = search_input 
    search_output = ''
    valid_search = False

    if search_type == "Please select an option":
        #user must select a category before inputting data
        Music_Search.children[1].value = "Invalid input. Please select a type" 
    else:
        for category in music_categories:
            if category == search_type:
                index = music_categories.index(category)
                #used to compare input data to the values in this index (category) of 'music_details'

        for song in music_details:
            available = True
            song_string = ""
            for song_details in song:
                song_string += (song_details + " | ")
            if search == song[index]:
                valid_search = True
                for rental in rental_details:
                    if song[0] in rental and rental[2] == "":
                        #checks if the song is currently being rented
                        available = False
                        break
                if available == True:
                    song_string += "AVAILABLE"
                else:
                    song_string += 'NOT AVAILABLE'
                search_output += song_string + "\n"
        if valid_search == True:
            Music_Search.children[1].value = search_output
            #directly alters value of 'music_search_output' in the 'Music_Search' widget
        else:
            Music_Search.children[1].value = "No existing search results"


    

## musicRent
#### Written by:F423894
#### Date: 05/12/24
### __Cell:__
musicRent defines the "music_rent()" function, which is used when the customer requests to rent a music record. It will use the functions defined in the database cell to check if the record referenced by the customers input exists, and that it is available to be rented This is done by iterating through the lists created by the functions in the "database" cell. If both are true, then the "Rental.txt" file is updated with a new entry, and the record has been successfully rented.
### __Functions:__
#### music_rent(Music_Rent, cust_id, m_id):
Takes three parameters:
 - "Music_Rent" represents the widget that contains all other widgets used to display GUI in the "Music Rent" tab. Is used to directly alter the text box which displays to the user the status of their rent request (e.g. "Record rented" or "Record not available").
 - "cust_id" is used to represent the value of the "customer_id" widget. Customer ID is used to keep track on who has rented what, as each customer is only allowed to rent a certain number of records at a time.
 - "m_id" represents the value of the "record_id" widget. Record ID is used to verify that the value of "m_id" is in fact an existing record, and is available to rent. 

In [3]:
#music rent
def music_rent(Music_Rent, cust_id, m_id):
    """Checks the validity of users inputs before then renting an available record"""
    date = datetime.today().strftime("%Y-%m-%d")
    subscriptions = sm.load_subscriptions()

    music_categories, music_details = get_music_details()
    rental_categories, rental_details = get_rental_details()
    
    customer_id = cust_id
    record_id = m_id
    customer_count = 0 
    #tracks the number of records the customer is currently renting
    valid_customer = sm.check_subscription((customer_id), subscriptions)
    valid_record = False
    available = True

    if valid_customer == False:
        Music_Rent.children[1].value = "Invalid customer ID"
    else:
        for song in music_details:
            if record_id in song:
                valid_record = True
        if valid_record == False:
            Music_Rent.children[1].value = "Invalid music ID"
        else:
            subscription_details = subscriptions.get(customer_id)
            sub_type = subscription_details.get("SubscriptionType")
            customer_limit = sm.get_rental_limit(sub_type)
            
            for rental in rental_details:
                #checking currently active rentals
                if customer_id in rental and rental[2] == "":
                    customer_count += 1
                if record_id in rental and rental[2] == "":
                    available = False
            if available == False:
                Music_Rent.children[1].value = "Record not available"
            else:
                if customer_count < customer_limit:
                    f = open("Rental.txt", "a")
                    rental_string = record_id + "," + date + "," + "" + "," + customer_id + "\n"
                    f.write(rental_string)
                    f.close()
                    Music_Rent.children[1].value = "Rental complete.\nEnter customer ID and record ID in the boxes above"
                else:
                    Music_Rent.children[1].value = "You have exceeded your rental limit"

## musicReturn
#### Written by:F423894
#### Date: 07/12/24
### __Cell:__
musicReturn defines the "music_return()" function. This is called when a customer requests to return a record they are renting. The ID of the record they are returning is checked to see if the record exists, and if the record is currently being rented. This is done by iterating through the list of records and rentals (using the functions from "database"). If both are true, then the record is returned, "Rental.txt" is updated and the record is now available for renting again. It also defines the "music_feedback()" function, which is called after a customer has successfully returned a record. The customer can opt to give feedback on the record in the form of a comment and a rating form 1-5. Each time feedback is submitted for a record, the "Music_Feedback.txt" file is updated with the new entry.
### __Functions:__ 
#### music_return(Music_Return, rec_id):
Takes two parameters:
 - "Music_Return" represents the widget which contains all other widgets used to display the GUI for the "Music Return" tab. Is used within "music_return()" to directly change the value of the text box responsible for displaying the status of the customers return request (similar to "Music_Rent" for the "music_rent()" function).
 - "rec_id" represents the ID of the record the customer wishes to return. This is used to check the ID is valid, meaning the record actually exists, and that the record is able to be returned (must be currently rented). 
#### music_feedback(Music_Return, rec_id, record_rating, record_review):
Takes four parameters:
 - "Music_Return" is explained above
 - "rec_id" is explained above, and its value is now stored as part of an entry in the "Music_Feedback.txt" file.
 - "record_rating" contains the numerical score from 1-5 given by the customer. Is also stored as part of an entry in the "Music_Feedback.txt" file.
- "record_review" contains the comment written by the customer which is also stored as part of an enrty in "Music_Feedback.txt", like the parameters above.

In [4]:
#music return
def music_return(Music_Return, rec_id):
    """Checks the validity of the users input and returns the rented record"""
    return_date = datetime.today().strftime("%Y-%m-%d")
    rental_categories, rental_details = get_rental_details()

    record_id = rec_id
    valid_record = False
    record_index = 0 
    for rental in rental_details:
        if record_id != "" and record_id in rental and rental[2] == "":
            valid_record = True
            break
        record_index += 1

    if valid_record == True:
        rental_string = ""
        rental_details[record_index][2] = return_date

        for dtls in rental_details[record_index]:
            rental_string += dtls
            if dtls != rental_details[record_index][-1]:
                rental_string += ","
        rental_string += "\n"

        f = open("Rental.txt", "r")
        details = f.readlines()
        details[record_index + 1] = rental_string
        #+1 to index due to first line being category names
        f.close()
        f = open("Rental.txt", "w")
        f.writelines(details)
        f.close()
        (Music_Return.children[0]).children[1].value = "Record returned"
    else:
        (Music_Return.children[0]).children[1].value = "Record ID is invalid or Record has already been returned"

def music_feedback(Music_Return, rec_id, record_rating, record_review):
    """Adds the users feedback to the 'Music_Feedback' file"""
    record_id = rec_id
    rating = record_rating
    review = record_review
    fm.add_feedback(record_id, rating, review, "Music_Feedback.txt")
    (Music_Return.children[0]).children[1].value = "Feedback submitted"

## InventoryPruning
#### Written by:F423894
#### Date: 09/12/24
### __Cell__:
InventoryPruning defines the "inventory_pruning()" function, which is called every time the user clicks the "Prune Inventory" button (see "menu.ipynb" cell). The function will iterate through every entry in the "Rental.txt" file, and count how many times each record appears. Results are stored in lists, and are used both to display records deemed unpopular and for use in a graph.
### __Functions:__
Takes one parameter:
 - "Inventory_Pruning" represents the widget which contains all widgets used to display the GUI for the "Inventory Pruning" tab in the menu. It is used to directly access and change the value of the "unpopular_records" widget, which displays the list of widgets deemed unpopular by the program. 

In [5]:
#inventory pruning
def invetory_pruning(Inventory_Pruning):
    """Calculates how many times each record has been rented and is part of displaying information visually"""
    rental_categories, rental_details = get_rental_details()
    rental_id_list = [] 
    rental_frequency_list = []
    added_ids = []
    #records are rented multiple times, so keeps track of which records have already been counted
    for rental in rental_details:
        rental_id_list.append(rental[0])
    for id in rental_id_list:
        if id not in added_ids:
            rental_frequency_list.append((id,rental_id_list.count(id)))
            added_ids.append(id)
    rental_frequency_list.sort()
    rental_frequency_list.reverse() 
    #used to display records alphabetically on the graph

    total_rentals = len(rental_details)
    total_records = len(rental_frequency_list)
    avg = total_rentals // total_records
    unpopular_records = "Here is a list of underperforming records:\n"
    for rental in rental_frequency_list:
        if rental[1] < avg:
            unpopular_records += rental[0] + ","
    unpopular_records = unpopular_records.strip(unpopular_records[-1])
    #gets rid of the last ',' at the end of the string
    (Inventory_Pruning.children[0]).children[0].value = unpopular_records

    id_list = []
    count_list = []
    for i in rental_frequency_list:
        id_list.append(i[0])
        count_list.append(i[1])
    return id_list, count_list


## menu.ipynb
#### Written by:F423894
#### Date: 10/12/24
### __Cell:__
The "menu.ipynb" cell is used to create and display all the widgets used to create the GUI the store manager can interact with. For each components of the program (Music Search, Music Rent etc.), there are dedicated functions to constantly checking for and detecting button clicks, inputs in text boxes and selections from dropdown windows. The widgets are combinded together and formatted in an easy to read/understand way using the "Hbox" and "Vbox" functions. Also within this cell is the Output widget called "display_graph", which is used to display the graph plotted using the "matplotlib" library and the values returned from the "inventory_pruning()" function.

In [6]:
#MUSIC SEARCH
select_type = widgets.Dropdown(
    options = ["Please select an option", "Title", "Artist", "Genre", "Medium"],
    value = "Please select an option",
    disabled = False
)
search_prompt = widgets.Label(
    value = "Would you like to search by Title, Artist, Genre or Medium? "
)
search_type = widgets.HBox(
    [search_prompt, select_type]
)
search_input = widgets.Text(
    placeholder = "Select an option above" 
)
search_inputs = widgets.VBox(
   [search_type, search_input] 
)
music_search_button = widgets.Button(
    description = "Search"
)
search = widgets.VBox(
    [search_inputs, music_search_button]
)
music_search_output = widgets.Textarea(
    placeholder = "Search results will appear here",
    layout = widgets.Layout(
        width = "800px",
        height = "30px"
    ),
    disabled = True
)
Music_Search = widgets.VBox(
    [search, music_search_output]
)

def select_type_change(type):
    """Detects changes in the 'select_type' dropdown box"""
    if select_type.value != "Please select an option":
        search_input.placeholder = (f"Enter the {select_type.value} you wish to search for: ")
    else:
        search_input.placeholder = "Select an option above"

def search_input_change(text):
    """Detects changes and updates the value stored in the 'search_input' text box widget"""
    search_input.value = text["new"]

def clicked_search(button):
    """Detects when the 'Search' button is clicked and calls the function to search for records"""
    music_search(Music_Search, select_type.value, search_input.value)

    search_input.value = ""
    if select_type.value != "Please select an option":
        search_input.placeholder = (f"Enter the {select_type.value} you wish to search for: ")
    else:
        search_input.placeholder = "Select an option above"
    #resets the input widgets back to their original state once a search has been made
    line_count = music_search_output.value.count("\n")
    if line_count > 1:
        music_search_output.layout.height = (f"{line_count * 18}px")
    else:
        music_search_output.layout.height = "30px"


#MUSIC RENT
customer_id = widgets.Text(
    description = "Customer ID: ",
    placeholder = "Enter customer ID:"
)
record_id = widgets.Text(
    description = "Record ID",
    placeholder = "Enter record ID:"
)
rent_inputs = widgets.HBox(
    [customer_id, record_id]
)
rent_button = widgets.Button(
    description = "Rent"
)
rent = widgets.HBox(
    [rent_inputs, rent_button]
)
rental_output = widgets.Textarea(
    value = "Enter customer ID and record ID in the boxes above",
    layout = widgets.Layout(
        width = "550px",
        height = "60px"
    ),
    disabled = True
)
Music_Rent = widgets.VBox(
    [rent, rental_output]
)

def customer_id_input_change(text):
    customer_id.value = text["new"]

def record_id_input_change(text):
    record_id.value = text["new"]

def clicked_rent(button):
    music_rent(Music_Rent, customer_id.value, record_id.value)
    if rental_output.value == "Rental complete.\nEnter customer ID and record ID in the boxes above":
        customer_id.value = ""
        customer_id.placeholder = "Enter customer ID:"
        record_id.value = ""
        record_id.placeholder = "Enter record ID:"
    #resets values of widgets to their initial values


#MUSIC RETURN
return_id = widgets.Text(
    description = "Record ID",
    placeholder = "Enter record ID"
)
return_button = widgets.Button(
    description = "Return"
)
return_inputs = widgets.HBox(
    [return_id, return_button]
)
return_output = widgets.Text(
    value = "",
    disabled = True
)
return_record = widgets.HBox(
    [return_inputs, return_output]
)
record_rating = widgets.IntSlider(
    value = 1,
    min = 1,
    max = 5,
    description = "Rating:",
    disabled = True
)
return_review = widgets.Textarea(
    description = "Comments:",
    placeholder = "Enter any comments you had about the record",
    disabled = True
)
review_info = widgets.HBox(
    [return_review, record_rating]
)
feedback_button = widgets.Button(
    description = "Submit Feedback",
    disabled = True
)
review_inputs = widgets.HBox(
    [review_info, feedback_button]
)

Music_Return = widgets.VBox(
    [return_record, review_inputs]
)

def return_id_input_change(text):
    return_id.value = text["new"]

def clicked_return(button):
    music_return(Music_Return, return_id.value)
    if return_output.value == "Record returned":
        record_rating.disabled = False
        return_review.disabled = False
        feedback_button.disabled = False
    #the widgets used for giving feedback are only accessible once a record has been returned
    else:
        record_rating.disabled = True
        record_rating.value = 1
        return_review.disabled = True
        return_review.value = ""
        return_review.placeholder = "Enter any comments you had about the record"
        feedback_button.disabled = True

def feedback_comments_input_change(text):
    return_review.value = text["new"]
    
def record_rating_input_change(number):
    record_rating.value = number["new"]

def clicked_submit_feedback(button):
    music_feedback(Music_Return, return_id.value, record_rating.value, return_review.value)
    if return_output.value == "Feedback submitted":
        return_id.value = ""
        return_id.placeholder = "Enter record ID:"
    record_rating.disabled = True
    record_rating.value = 1
    return_review.disabled = True
    return_review.value = ""
    return_review.placeholder = "Enter any comments you had about the record"
    feedback_button.disabled = True
    #values are reset as once feedback has been submitted you cannot submit feedback for the same record again


#INVENTORY PRUNING
prune_button = widgets.Button(
    description = "Prune Inventory"
)
unpopular_records = widgets.Textarea(
    placeholder = "Press the 'Prune Inventory' button to find underperforming records",
    layout = widgets.Layout(
        width = "500px",
        height = "60px"
    ),
    disabled = True
)
prune_inputs = widgets.HBox(
    [unpopular_records, prune_button]
)
display_graph = widgets.Output(
    #is left blank as there is nothing to display until the 'Prune Inventory' button is pressed
)
Inventory_Pruning = widgets.VBox(
    [prune_inputs, display_graph]
)
with display_graph:
    print("Graph will appear here:")

def clicked_prune(button):
    id_list, count_list = invetory_pruning(Inventory_Pruning)
    display_graph.clear_output()
    with display_graph:
        highest_count = max(count_list)
        x_ticks_list = []
        for i in range(highest_count + 1):
            x_ticks_list.append(i)

        plt.figure(figsize = (10, 8))
        plt.barh(id_list, count_list, color = "firebrick")
        #horizontal bar used so that record ID names are easier to read
        plt.xlabel("Record Count")
        plt.ylabel("Record ID")
        plt.grid(axis = "x", linestyle = ":", alpha = 1)
        plt.xticks(x_ticks_list)
        plt.show()

select_type.observe(select_type_change, names = "value")
search_input.observe(search_input_change, names = "value")
music_search_button.on_click(clicked_search)
#checking for any changes in the Music Search section of widgets

customer_id.observe(customer_id_input_change, names = "value")
record_id.observe(record_id_input_change, names = "value")
rent_button.on_click(clicked_rent)

return_id.observe(return_id_input_change, names = "value")
return_button.on_click(clicked_return)
return_review.observe(feedback_comments_input_change, names = "value")
record_rating.observe(record_rating_input_change, names = "value")
feedback_button.on_click(clicked_submit_feedback)

prune_button.on_click(clicked_prune)

accordion = widgets.Accordion(
    children = [Music_Search, Music_Rent, Music_Return, Inventory_Pruning],
    titles = ("Music Search", "Music Rent", "Music Return", "Inventory Pruning")
)
store = widgets.Label(
    value = "Music Store",
    style = {"font_size" : "40px"},
    layout = widgets.Layout(
        width = "250px",
        height = "50px",
        padding = "10px"
    )
)
Music_Store = widgets.VBox(
    [store, accordion]
)
display(Music_Store)
